# tf.TensorArray ：TensorFlow 动态数组 

在部分网络结构，尤其是涉及到时间序列的结构中，我们可能需要将一系列张量以数组的方式依次存放起来，以供进一步处理。当然，在即时执行模式下，你可以直接使用一个 Python 列表（List）存放数组。不过，如果你需要基于计算图的特性（例如使用 @tf.function 加速模型运行或者使用 SavedModel 导出模型），就无法使用这种方式了。因此，TensorFlow 提供了 tf.TensorArray ，一种支持计算图特性的 TensorFlow 动态数组。

其声明的方式为：

- arr = tf.TensorArray(dtype, size, dynamic_size=False) ：声明一个大小为 size ，类型为 dtype 的 TensorArray arr 。如果将 dynamic_size 参数设置为 True ，则该数组会自动增长空间。

其读取和写入的方法为：

- write(index, value) ：将 value 写入数组的第 index 个位置；

- read(index) ：读取数组的第 index 个值；

除此以外，TensorArray 还包括 stack() 、 unstack() 等常用操作，可参考[文档](https://www.tensorflow.org/api_docs/python/tf/TensorArray) 以了解详情。

请注意，由于需要支持计算图， tf.TensorArray 的 write() 方法是不可以忽略左值的！也就是说，在图执行模式下，必须按照以下的形式写入数组：

这样才可以正常生成一个计算图操作，并将该操作返回给 arr 。而不可以写成：

一个简单的示例如下：

In [1]:
import tensorflow as tf

@tf.function
def array_write_and_read():
    arr = tf.TensorArray(dtype=tf.float32, size=3)
    arr = arr.write(0, tf.constant(0.0))
    arr = arr.write(1, tf.constant(1.0))
    arr = arr.write(2, tf.constant(2.0))
    arr_0 = arr.read(0)
    arr_1 = arr.read(1)
    arr_2 = arr.read(2)
    return arr_0, arr_1, arr_2

a, b, c = array_write_and_read()
print(a, b, c)

tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(2.0, shape=(), dtype=float32)
